1. 데이터 RDD 생성 map()
2. collect()
3. blue mnms 만 꺼내기
4. 전체 합계 구하기
5. 도시별 합계 구하기

In [1]:
from pyspark import SparkConf, SparkContext
# 스파크 환경 설정 객체 생성
conf = SparkConf().setMaster("local").setAppName("mnm-rdd-count")
spark = SparkContext(conf=conf).getOrCreate()

24/12/03 16:16:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/03 16:16:27 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/12/03 16:16:27 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
import os
directory = os.path.join(os.getcwd(), "data")
filename = "mnm_dataset.csv"
filepath = os.path.join(directory, filename)

lines = spark.textFile("file:///"+filepath.replace("\\", "/"))
lines

file:////home/lab15/src/src/data/mnm_dataset.csv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [3]:
lines.take(5)

['State,Color,Count', 'TX,Red,20', 'NV,Blue,66', 'CO,Blue,79', 'OR,Blue,71']

In [10]:
header = lines.first()
header
filtered_lines = lines.filter(lambda row: row !=header)
filtered_lines.take(10)

['TX,Red,20',
 'NV,Blue,66',
 'CO,Blue,79',
 'OR,Blue,71',
 'WA,Yellow,93',
 'WY,Blue,16',
 'CA,Yellow,53',
 'WA,Green,60',
 'OR,Green,71',
 'TX,Green,68']

In [27]:
mnm_datas = filtered_lines.map(lambda row: row.split(","))
mnm_datas.collect()[:4]

[['TX', 'Red', '20'],
 ['NV', 'Blue', '66'],
 ['CO', 'Blue', '79'],
 ['OR', 'Blue', '71']]

색깔 별 mnm 추출 / 필요: color 예시 : Blue / filter() = sql WHERE 절과 같음

In [39]:
colors_cnt = mnm_datas.map(lambda row: (row[1], int(row[2])))
colors_cnt.take(10)

[('Red', 20),
 ('Blue', 66),
 ('Blue', 79),
 ('Blue', 71),
 ('Yellow', 93),
 ('Blue', 16),
 ('Yellow', 53),
 ('Green', 60),
 ('Green', 71),
 ('Green', 68)]

In [30]:
blues = mnm_datas.filter(lambda row: row[1] =="Blue")
blues.collect()

[['NV', 'Blue', '66'],
 ['CO', 'Blue', '79'],
 ['OR', 'Blue', '71'],
 ['WY', 'Blue', '16'],
 ['AZ', 'Blue', '75'],
 ['CO', 'Blue', '52'],
 ['CO', 'Blue', '95'],
 ['CO', 'Blue', '98'],
 ['CA', 'Blue', '13'],
 ['NV', 'Blue', '50'],
 ['TX', 'Blue', '49'],
 ['CA', 'Blue', '34'],
 ['UT', 'Blue', '97'],
 ['AZ', 'Blue', '59'],
 ['OR', 'Blue', '29'],
 ['OR', 'Blue', '54'],
 ['CO', 'Blue', '52'],
 ['WA', 'Blue', '59'],
 ['WY', 'Blue', '59'],
 ['WY', 'Blue', '53'],
 ['WA', 'Blue', '42'],
 ['WA', 'Blue', '37'],
 ['NV', 'Blue', '49'],
 ['NM', 'Blue', '15'],
 ['CA', 'Blue', '39'],
 ['UT', 'Blue', '48'],
 ['AZ', 'Blue', '82'],
 ['CO', 'Blue', '14'],
 ['WA', 'Blue', '18'],
 ['CO', 'Blue', '89'],
 ['TX', 'Blue', '80'],
 ['WA', 'Blue', '84'],
 ['NM', 'Blue', '46'],
 ['AZ', 'Blue', '16'],
 ['TX', 'Blue', '68'],
 ['CA', 'Blue', '99'],
 ['WY', 'Blue', '80'],
 ['CO', 'Blue', '44'],
 ['CO', 'Blue', '90'],
 ['TX', 'Blue', '19'],
 ['CA', 'Blue', '65'],
 ['TX', 'Blue', '98'],
 ['NM', 'Blue', '98'],
 ['AZ', 'Bl

총합계 / 필요 count / 연산 : map

In [36]:
city_rdd.countByValue()

defaultdict(int,
            {('TX', 20): 107,
             ('NV', 66): 110,
             ('CO', 79): 102,
             ('OR', 71): 97,
             ('WA', 93): 120,
             ('WY', 16): 103,
             ('CA', 53): 119,
             ('WA', 60): 118,
             ('TX', 68): 114,
             ('NV', 59): 125,
             ('AZ', 95): 116,
             ('WA', 20): 116,
             ('AZ', 75): 109,
             ('OR', 72): 106,
             ('NV', 98): 123,
             ('WY', 45): 107,
             ('CO', 52): 112,
             ('TX', 94): 103,
             ('CO', 82): 97,
             ('CO', 12): 115,
             ('CO', 17): 104,
             ('OR', 16): 116,
             ('AZ', 46): 107,
             ('NV', 43): 105,
             ('NM', 15): 106,
             ('WA', 12): 82,
             ('OR', 13): 128,
             ('CO', 95): 113,
             ('WY', 63): 108,
             ('TX', 63): 108,
             ('WY', 48): 110,
             ('OR', 95): 107,
             ('WA', 75): 1

In [31]:
mnm_sum = mnm_datas.map(lambda row: int(row[2])).sum()
print(f'총 합계 : {mnm_sum}')

총 합계 : 5500035


도시별 합계 / 필요 state,count / 연산 : map row[0]을 추가

In [32]:
city_rdd = mnm_datas.map(lambda row: (row[0], int(row[2])))

In [33]:
city_rdd.take(3)

[('TX', 20), ('NV', 66), ('CO', 79)]

key State 별로 count : reduceByKey()

In [35]:
city_total_cnt = city_rdd.reduceByKey(lambda a,b: a+b)
city_total_cnt.collect()

[('TX', 556493),
 ('NV', 548477),
 ('CO', 556302),
 ('OR', 542169),
 ('WA', 557227),
 ('WY', 538975),
 ('CA', 561184),
 ('AZ', 546812),
 ('NM', 553454),
 ('UT', 538942)]

Join 해보기 key 값이 같아야 하는데 value가 같아서 작동 안됨

In [47]:
colors_cnt = mnm_datas.map(lambda row: (row[1], int(row[2])))
city_rdd = mnm_datas.map(lambda row: (row[0], int(row[2])))
print(colors_cnt.take(3))
print(city_rdd.take(3))
join_rdd = city_rdd.join(colors_cnt)
print(join_rdd.collect())

[('Red', 20), ('Blue', 66), ('Blue', 79)]
[('TX', 20), ('NV', 66), ('CO', 79)]


[]
